In [2]:
import pandas as pd
import time
import os
import requests

# LLaMA model and API config
llama_model = "llama3.1:70b"
ollama_url = "http://............./api/generate" #Replace with a valid URL
headers = {"Content-Type": "application/json"}

# System prompt for LLaMA
instructions = (
    "You will be shown a sentence with a placeholder [ ] followed by four options labeled A, B, C, and D.\n"
    "Select ONLY the one best-fitting option to complete the sentence.\n"
    "Your response must ONLY be one single letter: A, B, C, or D.\n"
    "Respond with ONLY a single letter: A, B, C, or D."
     "DO NOT provide explanations, reasoning, or any other text. ONLY one letter."
)

# LLaMA call with retry
def call_llama(prompt: str) -> str:
    payload = {
        "model": llama_model,
        "prompt": f"{instructions}\n\n{prompt}",
        "stream": False,
        "options": {"temperature": 0}
    }

    for attempt in range(3):
        try:
            response = requests.post(ollama_url, json=payload, headers=headers)
            response.raise_for_status()
            result = response.json()["response"].strip().upper()
            if result in ["A", "B", "C", "D"]:
                return result
            print(f"⚠️ Unexpected output: '{result}'")
            return "Invalid"
        except Exception as e:
            print(f"❌ Error: {e}. Retrying...")
            time.sleep(5)
    return "Error"

# Process a shuffled CSV file
def process_file_llama_shfl(input_file):
    df = pd.read_csv(input_file)
    output_file = input_file.replace(".csv", "_Shfl_llama3.1_70_output.csv")

    # Resume logic
    if os.path.exists(output_file):
        df_out = pd.read_csv(output_file)
        start_index = df_out["LLaMA Choice"].notna().sum()
    else:
        df["LLaMA Choice"] = ""
        df_out = df.copy()
        start_index = 0

    print(f"🚀 Starting from row {start_index} in {input_file}")

    for i in range(start_index, len(df_out)):
        row = df_out.iloc[i]
        prompt = (
            f"Domain: {row['Domain']}\n"
            f"Prompt: {row['Prompt']}\n"
            f"A: {row['D']}\n"
            f"B: {row['C']}\n"
            f"C: {row['B']}\n"
            f"D: {row['A']}\n"
            "Which option best completes the sentence?"
        )
        choice = call_llama(prompt)
        df_out.at[i, "LLaMA Choice"] = choice

        df_out.to_csv(output_file, index=False, encoding="utf-8-sig")
        print(f"✅ [{i+1}/{len(df_out)}] Saved choice: {choice}")

    print(f"🎉 Finished: {output_file}")
    return df_out

# Run on all shuffled files
for file in ["High - Shfl.csv", "Mid - Shfl.csv", "Low - Shfl.csv"]:
    process_file_llama_shfl(file)


🚀 Starting from row 0 in High - Shfl.csv
✅ [1/60] Saved choice: D
✅ [2/60] Saved choice: C
✅ [3/60] Saved choice: A
✅ [4/60] Saved choice: B
✅ [5/60] Saved choice: A
✅ [6/60] Saved choice: A
✅ [7/60] Saved choice: B
✅ [8/60] Saved choice: A
✅ [9/60] Saved choice: C
✅ [10/60] Saved choice: C
✅ [11/60] Saved choice: C
✅ [12/60] Saved choice: A
✅ [13/60] Saved choice: D
✅ [14/60] Saved choice: B
✅ [15/60] Saved choice: C
✅ [16/60] Saved choice: B
✅ [17/60] Saved choice: B
✅ [18/60] Saved choice: B
✅ [19/60] Saved choice: B
✅ [20/60] Saved choice: C
✅ [21/60] Saved choice: B
✅ [22/60] Saved choice: B
✅ [23/60] Saved choice: C
✅ [24/60] Saved choice: B
✅ [25/60] Saved choice: C
✅ [26/60] Saved choice: D
✅ [27/60] Saved choice: C
✅ [28/60] Saved choice: B
✅ [29/60] Saved choice: B
✅ [30/60] Saved choice: C
✅ [31/60] Saved choice: C
✅ [32/60] Saved choice: A
✅ [33/60] Saved choice: A
✅ [34/60] Saved choice: B
✅ [35/60] Saved choice: D
✅ [36/60] Saved choice: B
✅ [37/60] Saved choice: B
✅ [38/